In [1]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get("google_maps")

In [2]:
pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 6.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import requests

# Define the search parameters
location = '-12.045951295587582, -77.03056459170315'  # Coordinates for Lima, Perú
radius = 150000  # Search radius in meters
keyword = 'school'  # Change depending on the category

# Define the base URL for the Places API
search_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
details_url = "https://maps.googleapis.com/maps/api/place/details/json"

params = {
    'location': location,
    'radius': radius,
    'keyword': keyword,
    'key': GOOGLE_API_KEY  # Replace with your actual API key
}

# Function to get place details and reviews
def get_place_details(place_id):
    details_params = {
        'place_id': place_id,
        'fields': 'reviews',
        'key': GOOGLE_API_KEY
    }
    details_response = requests.get(details_url, params=details_params)
    details_result = details_response.json().get('result', {})
    return details_result.get('reviews', [])

# Function to handle paginated results
def fetch_places(params):
    data = []
    reviews = []
    while True:
        response = requests.get(search_url, params=params)
        results = response.json().get('results', [])
        next_page_token = response.json().get('next_page_token', None)

        for result in results:
            place = {
                'name': result.get('name'),
                'business_status': result.get('business_status'),
                'lat': result['geometry']['location']['lat'],
                'lng': result['geometry']['location']['lng'],
                'rating': result.get('rating'),
                'user_ratings_total': result.get('user_ratings_total'),
                'vicinity': result.get('vicinity'),
                'price_level': result.get('price_level'),
                'place_id': result.get('place_id'),
            }
            data.append(place)

            # Fetch place details to get reviews
            place_reviews = get_place_details(place['place_id'])
            for review in place_reviews:
                reviews.append({
                    'place_id': place['place_id'],
                    'author_name': review.get('author_name'),
                    'rating': review.get('rating'),
                    'text': review.get('text'),
                    'time': review.get('time')
                })

        if next_page_token:
            params['pagetoken'] = next_page_token
            # Esperar unos segundos antes de realizar la siguiente solicitud debido a limitaciones de la API de Google Places
            import time
            time.sleep(2)
        else:
            break

    return data, reviews

# Fetch data
data, reviews = fetch_places(params)

# Create DataFrames
df_places = pd.DataFrame(data)
df_reviews = pd.DataFrame(reviews)

# Show the DataFrames
print(df_places)
print(df_reviews)

                                                 name business_status  \
0                                   Miraflores School     OPERATIONAL   
1                                     Berkeley School     OPERATIONAL   
2                                     Católica School     OPERATIONAL   
3              International Christian School of Lima     OPERATIONAL   
4                                   I.E Monte Carmelo     OPERATIONAL   
5                        Little Angels Nursery School     OPERATIONAL   
6                                   Mark Twain School     OPERATIONAL   
7                          Saint Patricks High School     OPERATIONAL   
8                                San Silvestre School     OPERATIONAL   
9   Carmelitas New School | Colegio Privado en Bar...     OPERATIONAL   
10                              John Nash High School     OPERATIONAL   
11                                      Global School     OPERATIONAL   
12                      I.E.PRIVADA HOWARD SCHOOL A

In [4]:
# Crear un archivo Excel con dos hojas
with pd.ExcelWriter('school.xlsx', engine='xlsxwriter') as writer:
    df_places.to_excel(writer, sheet_name='Detalles del Lugar', index=False)
    df_reviews.to_excel(writer, sheet_name='Reseñas del Lugar', index=False)

print("Archivo Excel creado exitosamente.")

Archivo Excel creado exitosamente.
